# Higher Order Functions: Lambdas ad Parameter and Return Values

## Declaring Higher-Order Functions

**A higher-order function is a function thatt takes another function as an arguement or returns one**

For example the `filter` standard-library function takes a predicate function as a parameter and is therefore a higher-order function

```
list.filter{ x > 0 }
```


### Function Types

Lets declare some function variable first

``` kotlin
val sum = { x:Int, y: Int -> x + y }
val action = { println(42) }
```

In this case, this compiler infers that both the sum and action variables have function types but if were to declare there respective types explicity it would look like this.

``` kotlin
val sum = (Int, Int) -> Int = { x,y -> x + y }
val action: () -> Unit = { println(42) }
```

### Parameters names of function types

``` kotlin
fun performRequest(
    url: String,
    callback: (code: Int, content: String) -> Unit
){
    
}

performRequest(""){ code, content -> /**/ }
```


Parameter names don't affect type matching. When we declare a lambda, we don't have to use the same parameter names as the ones used in the function type declaration. But the names improves readability of the code and can be used in the IDE for completion.

### Calling function passed as arguments

In [1]:
fun twoAndThree(operation: (Int, Int) -> Int ){
    val result = operation(2,3)
    println("The result is $result")
}

twoAndThree{ a,b -> a * b }

The result is 6


### Default and Null Values For Parameter with function types

When we declare a parameter of a function type, we can also specify its default value.

``` kotlin
fun <T> Collection<T>.joinToString( 
    separator: String = ", ",         
    prefix: String = "",         
    postfix: String = "",
    transform: (T) -> String = { it.toString() } ): String 
```

Another approach is declaring a function as nullable 

``` kotlin
fun <T> Collection<T>.joinToString(
    seperator: String = ","
    prefix: String = "",
    postfix: String = "",
    transform: ((T) -> String)? = null): String
```

### Returning function from function

The requirement to return a function from another function doesn't come up as often as passing a function to another function, but it's still useful. For example calculating the cost of shipping depending on the selected shipping method. We can define a function that chooses the appropriate login variant and returns it as another function. 

In [3]:
enum class Delivery { STANDARD, EXPEDITED }

class Order(val itemCount: Int)

fun getShippingCostCalculator(delivery: Delivery): (Order) -> Double {
    if (delivery == Delivery.EXPEDITED){
        return { order -> 6 + 2.1 * order.itemCount }
    }
    
    return { order -> 1.2 * order.itemCount }
}

val calculator = getShippingCostCalculator(Delivery.EXPEDITED)
println("Shipping costs ${calculator(Order(3))}")

Shipping costs 12.3


In [9]:
// Another Example

data class Person(
    val firstName: String,
    val lastName: String,
    val phoneNumber: String?
)

class ContactListFilters {
    var prefix: String = ""
    var onlyWithPhoneNumber: Boolean = false

    fun getPredicate(): (Person) -> Boolean {
        val startsWithPrefix = { p: Person ->
            p.firstName.startsWith(prefix) || p.lastName.startsWith(prefix)
        }
        if (!onlyWithPhoneNumber) {
            return startsWithPrefix
        }

        return { startsWithPrefix(it) && it.phoneNumber != null }
    }
}

val contacts = listOf(Person("Dmitry", "Jemerov", "123-4567"),
Person("Svetlana", "Isakova", null))

val contactListFilters = ContactListFilters()

 with(contactListFilters) {
 prefix = "Dm"
 onlyWithPhoneNumber = true
}

println(contacts.filter(contactListFilters.getPredicate()))

[Person(firstName=Dmitry, lastName=Jemerov, phoneNumber=123-4567)]


### Removing duplication through lambdas

In [8]:
data class SiteVisit(
    val path: String,
    val duration: Double,
    val os: OS
)

enum class OS { WINDOWS, LINUX, MAC, IOS, ANDROID }

val log = listOf(
    SiteVisit("/", 34.0, OS.WINDOWS),
    SiteVisit("/", 22.0, OS.MAC),
    SiteVisit("/login", 12.0, OS.WINDOWS),
    SiteVisit("/signup", 8.0, OS.IOS),
    SiteVisit("/", 16.3, OS.ANDROID)
)

// Lets say i want to find the average duration from windows machine
    
val averageWindowsDuration = log
    .filter{ it.os == OS.WINDOWS }
    .map(SiteVisit::duration)
    .average()

println(averageWindowsDuration)

// Now suppose we want to calculate same for the mac users. rather than duplicating
// the same code we extract the platform as the parameter.

fun List<SiteVisit>.averageDurationFor(os: OS) = 
    filter { it.os == os }.map(SiteVisit::duration).average()

println(log.averageDurationFor(OS.WINDOWS))
println(log.averageDurationFor(OS.MAC))

// Now suppose we want to find the average duration for the visit of Mobile Platform
// We could do something like

val averageMobileDuration = log
        .filter { it.os in setOf(OS.IOS, OS.ANDROID) }
        .map(SiteVisit::duration)
        .average()

println(averageMobileDuration)

23.0
23.0
22.0
12.15


## INLINE FUNCTION: REMOVING THE OVERHEAD OF LAMBDAS

Whenever we use a lambda expresion, an extra class is created; and if the lambda captures some variables, then a new object is created on every invocation. This introduces runtime overhead, causing an implementation that uses a lambda to be less efficient than a function that executes the same code directly

The solution to remedy this overhead is `inline` keyword,When we `inline` keyword the compiler won't generate a function call when the function is used and instead will replace every call to the function with the actual code implementing the function.


### How Inlining works

When we declare a function as `inline`, its body is inlined in other words, its substituted directly instead of the function invocation. Let's look at an example to understand the resulting code.

In [ ]:
inline fun <T> synchronized(lock: Lock, action: () -> T): T {
    lock.lock()
    try{
        return action()
    }
    finally {
        lock.unlock()
    }
}

val l = Lock()
synchronized(l){
    
}

The above function ensures that a shared resource isn't accessed concurrently by multiple threads. The function locks a Lock object, executes the given block of code, and then releases the lock.

Because we have declared the `synchronized` function as `inline`, the code generated for every call to it is the same as for `synchrnoized` statement in Java. 

``` kotlin
fun foo(l: Lock){
    println("Before sync")
    synchronized(l) {
        println("Action")
    }
    println("After sync")
}
```
Above code will compiled to

<img src="../resources/inline-example-1.png"/>

Note that the inlining is applied to the lambda expression as well as the implementation of the `synchronized` function. The bytecode generated from the lambda becomes the part of the definition of the calling function and isn't wrapped in an anonymous class implementing a function interface

### Restriction on inline function

Due to the way inlining is performed, not every function that uses lambdas can be inlined. When the function is inlined, the body of the lambda expression that's passed as an argument is substituted directly into the resulting code.
That restricts the possible uses of the corresponding parameter in the function body. If theis parameter is called, such code can be easily inlined. But if the parameter is stored somewhere for further use, the code of the lambda expression can't be inlined, because there must be an object that contains this code.

Generally, the parameter can be inlined if, its called directly or passed as an arguement to another `inline` function. Otherwise, the compiler will prohibit the inlining of the parameter with an error message that says "Illegal usage of inline-parameter"

``` kotlin 
fun <T, R> Sequence<T>.map(transform: (T) -> R): Sequence<R> {
    return TransformingSequence(this, transform)
}
```

the map function doesn't call the function passed as the `transform` parameter directly. Instead, it passes this function to the constructor of a class that stores it in a property. To support that, the lambda passed as the `transform` arguement needs to be compiled into the standard non-inline representation, as an anonymous class implementing a function interface.



### Deciding when to declare function as inline

Using the `inline` keyword can imporve performance only with functions that take lambdas as parameters

For regular function calls, the Java virtual machine already provides powerful inlining support. It analyzes the execution of your code and inlines calls whenever doing so provides the most benefit. This happens automatically and at the machine-code level. In bytecode, the implementation of each function is repeated only once and doesn’t need to be copied every place where the function is called, as with Kotlin’s inline functions. What’s more, the stacktrace is clearer if the function is called directly.


On the other hand, inlining functions with lambda parameters is beneficial. First, the overhead you avoid through inlining is more significant. You save not only on the call, but also on the creation of the extra class for each lambda and an object for the lambda instance. Second, the JVM currently isn’t smart enough to always perform inlining through the call and lambda. Finally inlining lets you use feature that are impossible to make work with regular lambdas, such as non-local returns



